In [1]:
def load_pickle(fname):
    with open(fname, 'rb') as f:
        return pickle.load(f)

## time
def aexp2zred(aexp):
    return [1.0/a - 1.0 for a in aexp]

def zred2aexp(zred):
    return [1.0/(1.0 + z) for z in zred]

def lbt2aexp(lts):
    import astropy.units as u
    from astropy.cosmology import WMAP7, z_at_value
    zreds = [z_at_value(WMAP7.lookback_time, ll * u.Gyr) for ll in lts]
    return [1.0/(1+z) for z in zreds]

def density_map(x, y, sort=True):
    from scipy.stats import gaussian_kde
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy) 
    z /= max(z)

    idx = z.argsort()    
    xx, yy = x[idx], y[idx]
    z = z[idx]
    
    return xx, yy, z


def sigma_clip_ind(c, high, low):
    """
        returns indices of sigma-clipping-safe elements.
    """
    import numpy as np
    ind = (np.mean(c) - np.std(c)*low < c) * (c < np.mean(c) + np.std(c)*high)
    return ind


def mask_outlier(y, low=1.5, high=1.5):
    """
        maks outlier assuming monotonic trend.
    """
    x = np.arange(len(y))

    # linear fitting .. more desirably, a very strong smoothing scheme that can reconstrcut mild curve.
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x,y)

    # extract linear fit
    yy = y - (slope * x + intercept)

    # sigma clipped value = mean of the rest 
    i_good = sigma_clip_ind(yy, low, high)
    yy[~i_good] = np.mean(yy[i_good])

    # add linear fit again
    return yy + (slope * x + intercept)


def smooth(x, beta=5, window_len=20, monotonic=False):
    """ 
    kaiser window smoothing 
    beta = 5 : Similar to a Hamming
    """
    
    if monotonic:
        """
        if there is an overall slope, smoothing may result in offset.
        compensate for that. 
        """
        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y=np.arange(len(x)))
        xx = np.arange(len(x)) * slope + intercept
        x = x - xx
    
    # extending the data at beginning and at the end
    # to apply the window at the borders
    s = np.r_[x[window_len-1:0:-1], x, x[-1:-window_len:-1]]
    w = np.kaiser(window_len,beta)
    y = np.convolve(w/w.sum(), s, mode='valid')
    if monotonic: 
         return y[int(window_len)/2:len(y)-int(window_len/2) + 1] + xx#[x[window_len-1:0:-1],x,x[-1:-window_len:-1]]
    else:
        return y[int(window_len)/2:len(y)-int(window_len/2) + 1]
        #return y[5:len(y)-5]

        
class MainPrg():
    import tree.ctutils as ctu
    import numpy as np
    
    def __init__(self, treedata, final_gal, nout_ini=None, nout_fi=None):

        temp_tree = ctu.extract_main_tree(treedata, final_gal)
        if nout_ini == None:
            nout_ini = min(temp_tree['nout'])
        if nout_fi == None:
            nout_fi = max(temp_tree['nout'])            
            
        self.nouts = np.arange(nout_fi, nout_ini -1, -1)
        self.idxs = temp_tree['id'] # nout_ini, nout_fi consideration needed.
        self.ids = temp_tree['Orig_halo_id']
        self.data = None
    
    def set_data(self, cat, nout):
        """
        compile data from catalogs.
        """
        if nout in self.nouts:
            # Declare self.data first if there isn't.
            if self.data == None:
                self.data = np.zeros(len(self.nouts), dtype=cat.dtype)
            inow = self.nouts == nout
            a = np.where(cat['idx'] == self.idxs[inow])[0]
            if len(a) > 0:
                self.data[inow] = cat[a]        
            else:
                pass
                #print(self.ids[inow],cat['id'])
        else:
            pass
            #print("No {} in the catalog".format(nout))
            
    def clip_non_detection(self):
        # end of galaxy tree = last non-zero position.
        # Note that 'id' can be 0 if phantom. But phantom is a valid datapoint
        i_first_nout = max(np.where(self.data['idx'] > 0)[0])
        #print('i_first', i_first_nout)
        # then, only [0-i_first_nout] are valid.
        # earlier then 187 - 91-th are zero. so get rid of them.
        self.data = self.data[:i_first_nout].copy()
        self.nouts = self.nouts[:i_first_nout].copy()
        self.ids = self.ids[:i_first_nout].copy()
        self.idxs = self.idxs[:i_first_nout].copy()
        
    def fill_missing_data(self):
        assert (self.ids[-1] != 0)
        
        # loop over all fields except id, index, and non-physical entries.
        i_bad = np.where(self.data['idx'] == 0)[0]
        for field in self.data.dtype.names:
            # do not modify index and id fields.
            if field in ["index", "id", "idx"]:
                continue
            arr = self.data[field] # it's a view.

            for i_b in i_bad:
                # neighbouring array might also be empty. Search for closest valid element.
                # left point
                i_l = i_b - 1
                while(i_l in i_bad):
                    i_l = i_l - 1
                # right point
                i_r = i_b + 1
                while(i_r in i_bad):
                    i_r = i_r + 1

                arr[i_b] = (arr[i_b -1] + arr[i_b +1])/2.
    

In [2]:
def fixed_ind_Lr(gal):
    nnouts = len(gal.nouts)
    ind_reff_fix = np.zeros(nnouts, dtype='i4')

    #print(gal.data['rgal'])
    smooth_r = smooth(mask_outlier(gal.data['rgal'], 1.5, 1.5), 50, monotonic=False)

    # fixed Reff array
    for i in range(nnouts):
        # 1Reff = 5 points
        reff_real = smooth_r[i]
        reff = gal.data['rgal'][i]
        try:
            ind_reff_fix[i] = np.round(reff_real/reff * 5) -1
        except:
            pass
    return ind_reff_fix


def smoothed_reff(cat, nout_merger):
    """
    returns "representative" lambda at each nout by assuming monotonic change in Reff. 
    During merger, Reff can fluctuate, and if has no physical meaning to infer Labda at Reff during merger stage. 
    So Reff' is derived by linear interpolating Reffs before and after the merger. 
    
    cat is one galaxy catalog over time.
    """
    import utils.match as mtc
    i_merger = np.where(cat['nout'] == nout_merger)[0]
    ind_lower = 20
    ind_upper = 20
    
    reffs = cat['rgal']
    # left and right values chosen by sigma-clipping
    r_lefts, b, c = scipy.stats.sigmaclip(reffs[max([0,i_merger-ind_lower]):i_merger], sig_lower, sig_upper)
    #print(r_lefts)
    r_left = r_lefts[-1]
    i_left = np.where(reffs == r_left)[0]
    

    r_rights, b,c = scipy.stats.sigmaclip(reffs[i_merger:min([i_merger+ind_upper,len(reffs)])], sig_lower, sig_upper)
    r_right = r_rights[0]
    i_right = np.where(reffs == r_right)[0]

    r_prime = reffs
    #print("chekc")
    #print(r_prime)
    r_prime[i_left : i_right + 1] = np.linspace(r_left, r_right, i_right - i_left + 1)
    return r_prime    

In [3]:
import numpy as np
import scipy.stats
import tree.ctutils as ctu
import matplotlib.pyplot as plt

# Read a single galaxy evolution catalog.
import pickle

In [4]:
clusters = ['36413', '05427'][:2]
# parameters used for lambda_arr clipping.
ind_upper = 20
ind_lower = 20
sig_upper = 2.0
sig_lower = 2.0

nout_ini = 60
nout_fi = 187

bad = 0

In [6]:
cdir = 'catalog_GM/'

verbose=True

ngals_tot = 0
for cluster in clusters:
    wdir = '/home/hoseung/Work/data/' + cluster + '/'
    # main galaxy list
    cat = pickle.load(open(wdir + cdir + 'catalog' + str(nout_fi) + '.pickle', 'rb'))
    ngals_tot = ngals_tot + len(cat['idx'])


nnouts = nout_fi - nout_ini + 1
lambda_evol_all = np.zeros([ngals_tot, nnouts])

mpgs = []
for cluster in clusters:
    print(cluster)
    wdir = '/home/hoseung/Work/data/' + cluster + '/'

    # Serialize catalogs. -> Only main galaxies
    # main galaxy list
    alltrees = ctu.load_tree(wdir, is_gal=True)
    ad = alltrees.data
    tn = ad[ad['nout'] == nout_fi]

    cat = pickle.load(open(wdir + cdir + 'catalog' + str(nout_fi) + '.pickle', 'rb'))
    #idx_all = [tn['id'][tn['Orig_halo_id'] == id_final][0] for id_final in cat['id']]
    idx_all = cat['idx']
    mpg_tmp =[MainPrg(ad, idx) for idx in idx_all]
    #print(mpgs[0].nouts)
    #print(mpgs[0].ids)
    for nout in range(nout_ini, nout_fi + 1):
        cat = pickle.load(open(wdir + cdir + 'catalog' + str(nout) + '.pickle', 'rb'))
        for gal in mpg_tmp:
            gal.set_data(cat, nout)

    while len(mpg_tmp) > 0:
        mpgs.append(mpg_tmp.pop())

    print("done")
#mpgs = (x for y in mpgs for x in y) # similar to flatten()


36413
Loaded an extended tree
done
05427
Loaded an extended tree


/usr/local/lib/python3.4/dist-packages/IPython/kernel/__main__.py:110: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


IndexError: index 0 is out of bounds for axis 0 with size 0

In [6]:
#mpgs = (x for y in mpgs for x in y) # similar to flatten()
for igal, gal in enumerate(mpgs):
    try:
        gal.clip_non_detection()
        gal.fill_missing_data()
        ind_reff_fix = fixed_ind_Lr(gal)

    except:
        continue

    ind_max = len(gal.data['lambda_arr'][0]) - 1

    for inout, ind in enumerate(ind_reff_fix):
        if ind == 0 : print(ind)
        lambda_evol_all[igal][inout] = gal.data['lambda_arr'][inout][min([ind_max,ind])]

/usr/lib/python3/dist-packages/scipy/stats/stats.py:3025: RuntimeWarning: invalid value encountered in sqrt
  t = r*np.sqrt(df/((1.0-r+TINY)*(1.0+r+TINY)))
/usr/lib/python3/dist-packages/scipy/stats/stats.py:3027: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
/usr/lib/python3/dist-packages/scipy/stats/stats.py:3029: RuntimeWarning: invalid value encountered in double_scalars
  sterrest = np.sqrt((1-r*r)*ssym / ssxm / df)
/home/hoseung/.local/lib/python3.4/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/hoseung/.local/lib/python3.4/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [7]:
#mpgs = (x for y in mpgs for x in y) # similar to flatten()
for igal, gal in enumerate(mpgs):
    try:
        gal.clip_non_detection()
        gal.fill_missing_data()
        #ind_reff_fix = fixed_ind_Lr(gal)

    except:
        continue

    #ind_max = len(gal.data['lambda_arr'][0]) - 1

    for inout in range(len(gal.nouts)):
        if ind == 0 : print(ind)
        lambda_evol_all[igal][inout] = gal.data['lambda_r'][inout]

In [8]:
zreds=[]
aexps=[]
import load
for nout in range(nout_ini, nout_fi+1):
    info = load.info.Info(nout=nout, base=wdir, load=True)
    aexps.append(info.aexp)
    zreds.append(info.zred)
aexps = np.array(aexps)
zreds = np.array(zreds)

# For a given list of nouts, 
# calculate a nice-looking set of zreds AND lookback times
z_targets=[0, 0.2, 0.5, 1, 2, 3]
z_target_str=["{:.2f}".format(z) for z in z_targets]
a_targets_z = zred2aexp(z_targets)
z_pos =  [nout_ini + (1 - (max(aexps) - a)/aexps.ptp()) * nnouts for a in a_targets_z]

lbt_targets=[0.00001,1,3,5,8,12]
lbt_target_str=["{:.0f}".format(l) for l in lbt_targets]
a_targets_lbt = lbt2aexp(lbt_targets)
lbt_pos = [nout_ini + (1 - (max(aexps) - a)/aexps.ptp()) * nnouts for a in a_targets_lbt]

In [9]:
lambda_range=[0., 0.8]
yticks_ok=[0.0, 0.2, 0.4, 0.6, 0.8]
nbins = 20
den_map = np.zeros((nbins, nnouts))

for i in range(nnouts):
    den_map[:,i], ypoints = np.histogram(lambda_evol_all[:,i], bins=nbins, range=lambda_range)
    den_map[:,i] /= den_map[:,i].max()

  

In [10]:
from astropy.stats import sigma_clip



nouts = np.arange(nout_ini, nout_fi + 1)
xx = np.tile(nouts, ngals_tot) 
all_data = lambda_evol_all.ravel()
data = all_data.copy()
data[np.isnan(data)] = 10
data[np.isinf(data)] = 10
data[data==0] = 10

#al, b1, c1 = scipy.stats.sigmaclip(data, 1.0, 1.0)
filtered_data = sigma_clip(data, sig=1.0, copy=True)
x = xx[~filtered_data.mask]
y = all_data[~filtered_data.mask]

xx,yy,z = density_map(x,y)




In [95]:
zreds

array([  1.73972577e+00,   1.70270268e+00,   1.66666662e+00,
         1.63157889e+00,   1.59740260e+00,   1.56410245e+00,
         1.53164542e+00,   1.49999994e+00,   1.46913579e+00,
         1.43902430e+00,   1.40963854e+00,   1.38095237e+00,
         1.35294102e+00,   1.32558128e+00,   1.29885047e+00,
         1.27272726e+00,   1.24719091e+00,   1.22222220e+00,
         1.19780216e+00,   1.17391291e+00,   1.15053756e+00,
         1.12765957e+00,   1.10526298e+00,   1.08333330e+00,
         1.06185564e+00,   1.04081632e+00,   1.02020202e+00,
         9.99999924e-01,   9.80198017e-01,   9.60784241e-01,
         9.41747460e-01,   9.23076797e-01,   9.04761848e-01,
         8.86792387e-01,   8.69158763e-01,   8.51851840e-01,
         8.34862321e-01,   8.18181764e-01,   8.01801651e-01,
         7.85714239e-01,   7.69911496e-01,   7.54385907e-01,
         7.39130432e-01,   7.24137901e-01,   7.09401681e-01,
         6.94915248e-01,   6.80672223e-01,   6.66666656e-01,
         6.52892510e-01,

In [11]:
fig, ax = plt.subplots(1)
im = ax.scatter(xx, yy, c=z, s=50, edgecolor='')
ax.set_ylim([-0.05, 0.9])
ax.set_yticks([0.0, 0.2, 0.4, 0.6, 0.8])
ax.set_yticklabels([str(yy) for yy in yticks_ok])

ax.set_xlabel("Redshift")
ax.set_xlim([nout_ini, nout_fi+1])
zz_target = [0, 0.2, 0.5, 1.0, 2.0, 3.0]
x_tick_pos = np.searchsorted(zreds[::-1], zz_target) + nout_ini# + nout_min

ax.set_xticks(x_tick_pos)#[::-1])
ax.set_xticklabels(labels = ["{:0.1f}".format(z) for z in zz_target])

plt.show()


/home/hoseung/.local/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [12]:

fig, ax = plt.subplots(1)
    
im = ax.imshow(den_map, origin="lower"#, cmap="Blues", interpolation="none"
               , extent=[0,nnouts,0,nbins], aspect='auto')

#ax.set_xlim([-1.5, lr_points*nreff])
ax.set_ylim([-0.5,nbins]) 
#ax.set_title(r"{:.1e} $< M_\ast <$ {:.1e}".format(mass_cut_l[imass], mass_cut_r[imass]))
#ax.text(2,17, "# gals:" + str(ngood)) # data coordinates

ax.set_yticks([0.5 + nbins * ly for ly in [0.0, 0.2, 0.4, 0.6, 0.8]])
ax.set_yticklabels([str(yy) for yy in yticks_ok])

ax.set_xlabel("Redshift")
#nout_min = 37
#ax.set_xlim([nout_min, 190])
#plt.gca().invert_xaxis()

# Redshift axis
#ax2 = ax.twiny()
zz_target = [0, 0.2, 0.5, 1.0, 2.0, 3.0]
x_tick_pos = np.searchsorted(zreds[::-1], zz_target)# + nout_min

ax.set_xticks(x_tick_pos)#[::-1])
ax.set_xticklabels(labels = ["{:0.1f}".format(z) for z in zz_target])

plt.show()